In [3]:
import pandas as pd
from sqlalchemy import create_engine, select
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import join
from models import Player, GameStats, Game
from db_config import get_database_engine

# Establish connection to the database
engine = get_database_engine()
Session = sessionmaker(bind=engine)

def fetch_player_game_log(player_name):
    session = Session()
    
    try:
        # Find the player by name
        player = session.query(Player).filter(Player.name == player_name).one_or_none()
        
        if not player:
            print(f"No player found with the name {player_name}.")
            return None

        # Build a SQL query to fetch the game stats and the corresponding game date
        # We join GameStats with Game on game_id to fetch the date of each game
        query = (
            select(
                GameStats,
                Game.date.label('game_date')  # This labels the column as 'game_date'
            )
            .select_from(
                join(GameStats, Game, GameStats.game_id == Game.id)
            )
            .where(GameStats.player_id == player.id)
        )

        # Use pandas to directly load SQL results into DataFrame
        df = pd.read_sql(query, engine)

        return df

    finally:
        session.close()


# Example usage
player_name = "Donovan Mitchell"
df = fetch_player_game_log(player_name)
if df is not None:
    print(df)


      id  player_id   game_id  points  assists  rebounds  steals  blocks  \
0   2482         53  22301174      33        5         4       4       0   
1   2483         53  22301158      29        8         4       3       1   
2   2484         53  22301127      10        7         5       3       2   
3   2485         53  22301110      24        2         3       2       2   
4   2486         53  22301077      13        6         0       1       0   
5   2487         53  22301067      12        8         3       2       0   
6   2488         53  22300967      13        2         3       1       0   
7   2489         53  22300951      14        5         4       0       1   
8   2490         53  22300846      19        3         5       0       0   
9   2491         53  22300832      31        6         7       0       1   
10  2492         53  22300820      16        5         4       1       0   
11  2493         53  22300781      30        7         6       2       0   
12  2494    